<h3 style="color:#2F4F4F"> 1.1. Problem Description</h3>

Consider a set of aircraft $\mathcal{P} = \{1,\dots,n\}$ that must land on one or more runways within specific time windows. Each aircraft $i$ becomes available to land (i.e., appears in the horizon) at time $A_i$, must land no earlier than $E_i$ (its earliest landing time) and no later than $L_i$ (its latest landing time), and ideally at the target time $T_i$. If an aircraft lands before its target time, it incurs an “early” penalty cost, and if it lands after its target time, it incurs a “late” penalty cost. Moreover, each pair of aircraft $(i,j)$ requires a minimum separation time $S_{ij}$ if $i$ lands before $j$ on the **same runway**. If there are multiple runways available, an aircraft may land on any runway, but safety regulations stipulate that the required minimum separation must be satisfied on whichever runway they share.

The goal is to determine a landing time (and runway assignment, if multiple runways exist) for each aircraft so as to minimize the total penalty cost of landing either before or after each aircraft’s target time, while respecting all time windows and minimum separation constraints.

<h4 style="color:#2F4F4F">Parameters</h4>
\begin{align}
n & \quad \text{Number of aircraft} \\
A_i & \quad \text{Appearance time (when aircraft $i$ arrives in the horizon)} \\
E_i & \quad \text{Earliest feasible landing time for aircraft $i$} \\
T_i & \quad \text{Target (preferred) landing time for aircraft $i$} \\
L_i & \quad \text{Latest feasible landing time for aircraft $i$} \\
\alpha_i & \quad \text{Penalty cost per unit time for landing before $T_i$} \\
\beta_i & \quad \text{Penalty cost per unit time for landing after $T_i$} \\
S_{ij} & \quad \text{Separation time required between landings of $i$ and $j$ on the same runway} \\
R & \quad \text{Number of available runways (if dealing with multiple-runway scenarios)} \\
\end{align}

<h4 style="color:#2F4F4F">Decision Variables</h4>
\begin{align*}
x_i & \quad \text{Landing time assigned to aircraft $i$} \\
\delta_{ij} & =
\begin{cases}
1, & \text{if aircraft $i$ lands before aircraft $j$ (on the same runway)} \\
0, & \text{otherwise}
\end{cases} \\
y_{ir} & =
\begin{cases}
1, & \text{if aircraft $i$ lands on runway $r$} \\
0, & \text{otherwise}
\end{cases} \quad (\text{Only needed for multiple-runway problems}) \\
e_i & \quad \text{Amount of time aircraft $i$ lands earlier than $T_i$ (early deviation)} \\
\ell_i & \quad \text{Amount of time aircraft $i$ lands later than $T_i$ (late deviation)} \\
\end{align*}

<h4 style="color:#2F4F4F">Objective Function and Constraints</h4>
\begin{align}
\text{Minimize} \quad 
& \sum_{i \in \mathcal{P}} \Big( \alpha_i \cdot e_i + \beta_i \cdot \ell_i \Big) \\
\text{subject to} \\
& x_i = T_i - e_i + \ell_i && \forall i \in \mathcal{P} \\
& E_i \;\;\leq\;\; x_i \;\;\leq\;\; L_i && \forall i \in \mathcal{P} \\
& e_i \geq 0,\quad \ell_i \geq 0 && \forall i \in \mathcal{P} \\
& x_j \;\geq\; x_i + S_{ij} - M \cdot (1 - \delta_{ij}) && \forall i \neq j, \ i,j \in \mathcal{P},\\
& \delta_{ij} + \delta_{ji} = 1 && \forall i \neq j, \ i,j \in \mathcal{P} \\
& \text{(If multiple runways)} \quad \sum_{r=1}^{R} y_{ir} = 1 && \forall i \in \mathcal{P} \\
& \text{(If multiple runways)} \quad x_j \;\geq\; x_i + S_{ij} \cdot \Big(y_{ir} \cdot y_{jr}\Big) - M \cdot (1 - \delta_{ij}) && \forall i \neq j,\; r = 1,\dots,R \\
& x_i, e_i, \ell_i \geq 0, \quad \delta_{ij}, y_{ir} \in \{0,1\}.
\end{align}


In [1]:
def read_data_from_file(filename):
    """
    Reads data from a file with the specified format for an air traffic scheduling problem.

    Args:
        filename (str): The path to the data file.

    Returns:
        tuple: A tuple containing the parsed data:
            - num_planes (int): The number of planes.
            - freeze_time (int): The freeze time.
            - planes_data (list): A list of dictionaries, where each dictionary contains
              the data for a plane.
            - separation_times (list of lists): A 2D list of separation times.
    """
    try:
        with open(filename, "r") as f:
            # Read the first line: number of planes and freeze time
            first_line = f.readline().strip().split()
            num_planes = int(first_line[0])
            freeze_time = int(first_line[1])

            planes_data = []
            separation_times = []
            
            for _ in range(num_planes):
                line = f.readline().strip().split()
                appearance_time = int(line[0])
                earliest_landing_time = int(line[1])
                target_landing_time = int(line[2])
                latest_landing_time = int(line[3])
                penalty_early = float(line[4])
                penalty_late = float(line[5])
                planes_data.append(
                    {
                        "appearance_time": appearance_time,
                        "earliest_landing_time": earliest_landing_time,
                        "target_landing_time": target_landing_time,
                        "latest_landing_time": latest_landing_time,
                        "penalty_early": penalty_early,
                        "penalty_late": penalty_late,
                    }
                )
                
                separation_row = []
                while len(separation_row) < num_planes:
                    line = f.readline().strip().split()
                    separation_row.extend([int(x) for x in line])

                separation_times.append(separation_row)

        return num_planes, freeze_time, planes_data, separation_times

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return None, None, None, None
    except ValueError:
        print(f"Error: Error reading data in file '{filename}'.")
        return None, None, None, None


In [14]:
filename = "data/airland1.txt"

num_planes, freeze_time, planes_data, separation_times = read_data_from_file(filename)

if num_planes is not None:
    print("Number of planes:", num_planes)
    print("Freeze time:", freeze_time)
    print("\nPlane data:")
    for i, plane in enumerate(planes_data):
        print(f"Plane {i+1}: {plane}")
    print("\nSeparation times:")
    for i, row in enumerate(separation_times):
        print(f"After plane {i+1}: {row}")

Number of planes: 10
Freeze time: 10

Plane data:
Plane 1: {'appearance_time': 54, 'earliest_landing_time': 129, 'target_landing_time': 155, 'latest_landing_time': 559, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 2: {'appearance_time': 120, 'earliest_landing_time': 195, 'target_landing_time': 258, 'latest_landing_time': 744, 'penalty_early': 10.0, 'penalty_late': 10.0}
Plane 3: {'appearance_time': 14, 'earliest_landing_time': 89, 'target_landing_time': 98, 'latest_landing_time': 510, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 4: {'appearance_time': 21, 'earliest_landing_time': 96, 'target_landing_time': 106, 'latest_landing_time': 521, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 5: {'appearance_time': 35, 'earliest_landing_time': 110, 'target_landing_time': 123, 'latest_landing_time': 555, 'penalty_early': 30.0, 'penalty_late': 30.0}
Plane 6: {'appearance_time': 45, 'earliest_landing_time': 120, 'target_landing_time': 135, 'latest_landing_time': 576, 'penalty_earl

In [15]:
from ortools.linear_solver import pywraplp


def interval_overlap(interval1, interval2):
    """
    Check if two intervals overlap.

    Args:
        interval1 (tuple): A tuple representing an interval (start, end).
        interval2 (tuple): A tuple representing an interval (start, end).

    Returns:
        bool: True if the intervals overlap, False otherwise.
    """

    start1, end1 = interval1
    start2, end2 = interval2

    return end1 > start2 and end2 > start1

In [16]:
# one runway
def create_or_tools_lp_model(num_planes, freeze_time, planes_data, separation_times):
    # Create the LP solver
    solver = pywraplp.Solver.CreateSolver("SAT")
    variables = {}

    # Decision Variables
    # x_i: Landing time for plane i
    # (1)
    landing_times = [
        solver.NumVar(
            planes_data[i]["earliest_landing_time"],
            planes_data[i]["latest_landing_time"],
            f"LandingTime_{i}",
        )
        for i in range(num_planes)
    ]
    variables["landing_times"] = landing_times

    # delta_ij:  Fraction representing if plane i lands before plane j (0 to 1)
    # Note: In a pure LP model, we relax the integrality constraint.
    landing_order = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                landing_order[(i, j)] = solver.NumVar(0, 1, f"LandingOrder_{i}_{j}")
    variables["landing_order"] = landing_order

    # alpha_i: Time by which plane i lands before its target time
    early_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["target_landing_time"]
                - planes_data[i]["earliest_landing_time"],
                0,
            ),
            f"EarlyDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["early_deviation"] = early_deviation

    # beta_i: Time by which plane i lands after its target time
    late_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["latest_landing_time"]
                - planes_data[i]["target_landing_time"],
                0,
            ),
            f"LateDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["late_deviation"] = late_deviation

    # Constraints
    # (2)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            solver.Add(landing_order[(i, j)] + landing_order[(j, i)] == 1)

    # Set V
    certain_with_no_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij > earliest_j:
                    certain_with_no_separation_pairs.append((i, j))

    # Set W
    certain_with_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij <= earliest_j:
                    certain_with_separation_pairs.append((i, j))

    uncertain_pairs = []

    # Set U
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j, latest_j = (
                    planes_data[j]["earliest_landing_time"],
                    planes_data[j]["latest_landing_time"],
                )

                if (
                    (earliest_j <= earliest_i <= latest_j)
                    or (earliest_j <= latest_i <= latest_j)
                ) or (
                    (earliest_i <= earliest_j <= latest_i)
                    or (earliest_i <= latest_j <= latest_i)
                ):
                    uncertain_pairs.append((i, j))

    # Enforce separation for pairs where order is determined (Set V)
    for i, j in certain_with_no_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1)
        solver.Add(landing_times[j] >= landing_times[i] + separation_times[i][j])

    # Enforce order for pairs where order is determined and separation is automatic (Set W)
    for i, j in certain_with_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1)

    M = sum(plane["latest_landing_time"] for plane in planes_data)

    for i, j in uncertain_pairs:
        solver.Add(landing_times[j] >= landing_times[i] + separation_times[i][j] - M * (landing_order[(j, i)]))

    # 4. Relating Deviation Variables to Landing Times
    for i in range(num_planes):
        earliest_i = planes_data[i]["earliest_landing_time"]
        latest_i = planes_data[i]["latest_landing_time"]
        target_i = planes_data[i]["target_landing_time"]

        # (14)
        solver.Add(early_deviation[i] >= target_i - landing_times[i])

        # (15)
        solver.Add(early_deviation[i] >= 0)
        solver.Add(early_deviation[i] <= target_i - earliest_i)

        # (16)
        solver.Add(late_deviation[i] >= landing_times[i] - target_i)

        # (17)
        solver.Add(late_deviation[i] >= 0)
        solver.Add(late_deviation[i] <= latest_i - target_i)

        # (18)
        solver.Add(
            landing_times[i] == target_i - early_deviation[i] + late_deviation[i]
        )

    # Objective Function: Minimize the cost of deviation from target landing times
    # objective_terms = []
    # for i in range(num_planes):
    #     objective_terms.append(planes_data[i]["penalty_early"] * early_deviation[i])
    #     objective_terms.append(planes_data[i]["penalty_late"] * late_deviation[i])
    # solver.Minimize(solver.Sum(objective_terms))
    
    objective = solver.Objective()

    for i in range(num_planes):
        objective.SetCoefficient(early_deviation[i], planes_data[i]["penalty_early"])
        objective.SetCoefficient(late_deviation[i], planes_data[i]["penalty_late"])

    objective.SetMinimization()

    return solver, variables

In [17]:
if num_planes is not None:
    solver, variables = create_or_tools_lp_model(
        num_planes, freeze_time, planes_data, separation_times
    )
    status = solver.Solve()

    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print("Status:", status)
        print("Optimal Cost:", solver.Objective().Value())
        print("\nOptimal Landing Times:")
        for i in range(num_planes):
            print(f"Plane {i+1}: {variables['landing_times'][i].solution_value()} - Target Time: {planes_data[i]['target_landing_time']}")

        print("\nLanding Order Variables (Note: May not be strictly 0 or 1 in LP):")
        for i in range(num_planes):
            for j in range(num_planes):
                if i != j:
                    print(
                        f"LandingOrder_{i}_{j}: {variables['landing_order'][(i, j)].solution_value()}"
                    )

    else:
        print("No optimal solution found.")
        print("Status:", status)
pywraplp.Solver.INFEASIBLE

Status: 0
Optimal Cost: 700.0

Optimal Landing Times:
Plane 1: 165.0 - Target Time: 155
Plane 2: 258.0 - Target Time: 258
Plane 3: 98.0 - Target Time: 98
Plane 4: 106.0 - Target Time: 106
Plane 5: 118.0 - Target Time: 123
Plane 6: 134.0 - Target Time: 135
Plane 7: 126.0 - Target Time: 138
Plane 8: 142.0 - Target Time: 140
Plane 9: 150.0 - Target Time: 150
Plane 10: 180.0 - Target Time: 180

Landing Order Variables (Note: May not be strictly 0 or 1 in LP):
LandingOrder_0_1: 1.0
LandingOrder_0_2: 0.0
LandingOrder_0_3: 0.0
LandingOrder_0_4: 0.0
LandingOrder_0_5: 0.0
LandingOrder_0_6: 0.0
LandingOrder_0_7: 0.0
LandingOrder_0_8: 0.0
LandingOrder_0_9: 1.0
LandingOrder_1_0: 0.0
LandingOrder_1_2: 0.0
LandingOrder_1_3: 0.0
LandingOrder_1_4: 0.0
LandingOrder_1_5: 0.0
LandingOrder_1_6: 0.0
LandingOrder_1_7: 0.0
LandingOrder_1_8: 0.0
LandingOrder_1_9: 0.0
LandingOrder_2_0: 1.0
LandingOrder_2_1: 1.0
LandingOrder_2_3: 1.0
LandingOrder_2_4: 1.0
LandingOrder_2_5: 1.0
LandingOrder_2_6: 1.0
LandingOrder

2

In [6]:
def multiple_runways_problem(num_planes, freeze_time, planes_data, separation_times, num_runways):
    # Create the LP solver
    solver = pywraplp.Solver.CreateSolver("SAT")  # Using the SAT solver
    variables = {}

    # Decision Variables
    # x_i: Landing time for plane i
    # (1)
    landing_times = [
        solver.NumVar(
            planes_data[i]["earliest_landing_time"],
            planes_data[i]["latest_landing_time"],
            f"LandingTime_{i}",
        )
        for i in range(num_planes)
    ]
    variables["landing_times"] = landing_times

    # delta_ij:  Fraction representing if plane i lands before plane j (0 to 1)
    # Note: In a pure LP model, we relax the integrality constraint.
    landing_order = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                landing_order[(i, j)] = solver.NumVar(0, 1, f"LandingOrder_{i}_{j}")
    variables["landing_order"] = landing_order

    # alpha_i: Time by which plane i lands before its target time
    early_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["target_landing_time"]
                - planes_data[i]["earliest_landing_time"],
                0,
            ),
            f"EarlyDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["early_deviation"] = early_deviation

    # beta_i: Time by which plane i lands after its target time
    late_deviation = [
        solver.NumVar(
            0,
            max(
                planes_data[i]["latest_landing_time"]
                - planes_data[i]["target_landing_time"],
                0,
            ),
            f"LateDeviation_{i}",
        )
        for i in range(num_planes)
    ]
    variables["late_deviation"] = late_deviation

    # s_ij: Separation time between plane i landing and plane j landing (where plane i lands before plane j and they land on different runways)
    separation_times_between_runways = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                separation_times_between_runways[(i, j)] = solver.NumVar(0, separation_times[i][j], f"SeparationTime_{i}_{j}")

    variables["separation_times_between_runways"] = separation_times_between_runways

    # z_ij: 1 if plane i and plane j land on the same runway, 0 otherwise
    same_runway = {}
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                same_runway[(i, j)] = solver.NumVar(0, 1, f"SameRunway_{i}_{j}")

    variables["same_runway"] = same_runway

    # y_ir: 1 if plane i lands on runway r, 0 otherwise
    landing_runway = {}
    for i in range(num_planes):
        for r in range(num_runways):
            landing_runway[(i, r)] = solver.NumVar(0, 1, f"LandingRunway_{i}_{r}")

    variables["landing_runway"] = landing_runway

    # Constraints
    # (2)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            solver.Add(landing_order[(i, j)] + landing_order[(j, i)] == 1)

    # Set W
    # (3)
    certain_with_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij <= earliest_j:
                    certain_with_separation_pairs.append((i, j))

    # Set V
    # (4)
    certain_with_no_separation_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j = planes_data[j]["earliest_landing_time"]
                separation_ij = separation_times[i][j]
                if latest_i < earliest_j and latest_i + separation_ij > earliest_j:
                    certain_with_no_separation_pairs.append((i, j))

    # Set U
    # (5)
    uncertain_pairs = []
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                earliest_i, latest_i = (
                    planes_data[i]["earliest_landing_time"],
                    planes_data[i]["latest_landing_time"],
                )
                earliest_j, latest_j = (
                    planes_data[j]["earliest_landing_time"],
                    planes_data[j]["latest_landing_time"],
                )

                if (
                    (earliest_j <= earliest_i <= latest_j)
                    or (earliest_j <= latest_i <= latest_j)
                ) or (
                    (earliest_i <= earliest_j <= latest_i)
                    or (earliest_i <= latest_j <= latest_i)
                ):
                    uncertain_pairs.append((i, j))

    # Enforce separation for pairs where order is determined (Set V)
    # (6) and (7)
    for i, j in certain_with_no_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1) # (6)
        solver.Add(landing_times[j] >= landing_times[i] + separation_times[i][j] * same_runway[(i, j)] + separation_times_between_runways[(i, j)] * (1 - same_runway[(i, j)])) # (7)

    # Enforce order for pairs where order is determined and separation is automatic (Set W)
    # (6)
    for i, j in certain_with_separation_pairs:
        solver.Add(landing_order[(i, j)] == 1) # (6)

    M = sum(plane["latest_landing_time"] for plane in planes_data)

    for i, j in uncertain_pairs:
        # solver.Add(landing_times[j] >= landing_times[i] + separation_times[i][j] * same_runway[(i, j)] + separation_times_between_runways[(i, j)] * (1 - same_runway[(i, j)]) - M * (landing_order[(j, i)])) # (8)
        latest_i = planes_data[i]["latest_landing_time"]
        earliest_j = planes_data[j]["earliest_landing_time"]
        
        solver.Add(
            landing_times[j]
            >= landing_times[i]
            + separation_times[i][j] * same_runway[(i, j)]
            + separation_times_between_runways[(i, j)] * (1 - same_runway[(i, j)])
            - (latest_i + max(separation_times[i][j], separation_times_between_runways[(i, j)]) - earliest_j) * (landing_order[(j, i)])
        )  # (8)

    # 4. Relating Deviation Variables to Landing Times
    for i in range(num_planes):
        earliest_i = planes_data[i]["earliest_landing_time"]
        latest_i = planes_data[i]["latest_landing_time"]
        target_i = planes_data[i]["target_landing_time"]

        # (14)
        solver.Add(early_deviation[i] >= target_i - landing_times[i])

        # (15)
        solver.Add(early_deviation[i] >= 0)
        solver.Add(early_deviation[i] <= target_i - earliest_i)

        # (16)
        solver.Add(late_deviation[i] >= landing_times[i] - target_i)

        # (17)
        solver.Add(late_deviation[i] >= 0)
        solver.Add(late_deviation[i] <= latest_i - target_i)

        # (18)
        solver.Add(
            landing_times[i] == target_i - early_deviation[i] + late_deviation[i]
        )

    # New constraints for multiple runways
    # (28)
    for i in range(num_planes):
        solver.Add(solver.Sum(landing_runway[(i, r)] for r in range(num_runways)) == 1)

    # (29)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            solver.Add(same_runway[(i, j)] == same_runway[(j, i)])

    # (30)
    for i in range(num_planes):
        for j in range(i + 1, num_planes):
            for r in range(num_runways):
                solver.Add(same_runway[(i, j)] >= landing_runway[(i, r)] + landing_runway[(j, r)] - 1)

    # Objective Function: Minimize the cost of deviation from target landing times
    # objective_terms = []
    # for i in range(num_planes):
    #     objective_terms.append(planes_data[i]["penalty_early"] * early_deviation[i])
    #     objective_terms.append(planes_data[i]["penalty_late"] * late_deviation[i])
    # solver.Minimize(solver.Sum(objective_terms))
    
    for i in range(num_planes):
        solver.SetCoefficient(early_deviation[i], planes_data[i]["penalty_early"])
        solver.SetCoefficient(late_deviation[i], planes_data[i]["penalty_late"])
    
    solver.SetMinimization()
    
    return solver, variables

In [7]:
solver, variables = multiple_runways_problem(
    num_planes, freeze_time, planes_data, separation_times, 2
)
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print("Status:", status)
    print("Optimal Cost:", solver.Objective().Value())
    print("\nOptimal Landing Times:")
    for i in range(num_planes):
        print(f"Plane {i+1}: {variables['landing_times'][i].solution_value()} - Target Time: {planes_data[i]['target_landing_time']}")

    print("\nLanding Order Variables (Note: May not be strictly 0 or 1 in LP):")
    for i in range(num_planes):
        for j in range(num_planes):
            if i != j:
                print(
                    f"LandingOrder_{i}_{j}: {variables['landing_order'][(i, j)].solution_value()}"
                )

else:
    print("No optimal solution found.")
    print("Status:", status)

TypeError: 